# HyDE Table Retrieval

Hypothetical Document Embeddings for table search:
1. Generate table descriptions using LLM
2. Generate hypothetical table descriptions from queries using LLM
3. Encode and retrieve using MiniLM-L6-v2 + FAISS

## Setup

In [ ]:
import pandas as pd
import numpy as np
import json
import faiss
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from tqdm import tqdm

from openai import OpenAI
from functools import reduce
import time

client = OpenAI(api_key="<insert key here>")


/opt/anaconda3/envs/smartcat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [2]:
# Load tables
tables_df = pd.read_csv('data/wikitables_mini.csv')
print(f"Loaded {len(tables_df)} tables")
print(f"Columns: {list(tables_df.columns)}")
tables_df.head(2)

Loaded 2932 tables
Columns: ['table_id', 'page_title', 'section_title', 'table_caption', 'headers', 'sample_data']


,table_id,page_title,section_title,table_caption,headers,sample_data
0,table-0001-249,Auburn Tigers swimming and diving,Summer Olympic Games Beijing 2008,Summer Olympic Games Beijing 2008,"[""Athlete"", ""Nation"", ""Total"", ""Gold"", ""Silver...","[[""[Fr\u00e9d\u00e9rick_Bousquet|Fr\u00e9d\u00..."
1,table-0001-400,Bisphenol A,Low-dose exposure in animals,Low-dose exposure in animals,"[""Dose (\u00b5g/kg/day)"", ""[Environmental_Work...","[[""0.025"", ""\""Permanent changes to genital tra..."


## Generate Table Descriptions (HyDE)

Use LLM to generate natural descriptions from table metadata.

In [ ]:
def generate_table_description_with_llm(row, max_chars=256):
    """
    Generate table description using LLM.
    
    """
    
    # Prepare table metadata as prompt context
    metadata = []
    if pd.notna(row['table_caption']):
        metadata.append(f"Caption: {row['table_caption']}")
    if pd.notna(row['page_title']):
        metadata.append(f"Page: {row['page_title']}")
    if pd.notna(row['section_title']):
        metadata.append(f"Section: {row['section_title']}")
    
    try:
        headers = json.loads(row['headers'])
        if headers:
            metadata.append(f"Columns: {', '.join([str(h) for h in headers[:10]])}")
    except:
        pass
    
    try:
        sample_data = json.loads(row['sample_data'])
        if sample_data and len(sample_data) > 0:
            sample_str = str(sample_data[0][:5])
            metadata.append(f"Sample: {sample_str}")
    except:
        pass
    
    metadata_str = '\n'.join(metadata)

    sleep_timer = 1
    time.sleep(sleep_timer)

    system_msg = 'You are a helpful assistant.'

    user_msg = f"""
Given the following table metadata, write a natural 
description of what this table contains in 1-2 sentences.
Keep it under 256 characters.\n\n{metadata_str}
    """

    response = client.chat.completions.create(model="gpt-4o",
    messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}],
    max_tokens=4000, 
    temperature=0.0)

    hypothetical_description = response.choices[0].message.content
    
    return hypothetical_description[:max_chars]

# Test
print("Example LLM-generated descriptions:")
print("=" * 80)
for i in range(3):
    desc = generate_table_description_with_llm(tables_df.iloc[i])
    print(f"\n{i+1}. {desc}")

Example LLM-generated descriptions:

1. The table lists Auburn Tigers swimmers and divers who participated in the 2008 Beijing Summer Olympics, detailing each athlete's nation, total medals won, and the count of gold, silver, and bronze medals across various events.

2. The table details the effects of low-dose Bisphenol A exposure in animals, listing the dose in µg/kg/day, observed effects, and the study year.

3. This table lists players from the Charlotte Bobcats' all-time roster, highlighting those selected in the 2004 NBA Expansion Draft and indicating which players are currently on the roster.


In [4]:
# Generate descriptions for all tables
print("Generating LLM descriptions for all tables...")
print("NOTE: This will make ~3K LLM API calls. Consider batch processing or caching.")
print()

table_descriptions = []
table_ids = []

for idx, row in tqdm(tables_df.iterrows(), total=len(tables_df)):
    table_descriptions.append(generate_table_description_with_llm(row))
    table_ids.append(row['table_id'])

print(f"Generated {len(table_descriptions)} descriptions")
print(f"Length stats - Mean: {np.mean([len(d) for d in table_descriptions]):.1f}, Max: {max([len(d) for d in table_descriptions])}")

Generating LLM descriptions for all tables...
NOTE: This will make ~3K LLM API calls. Consider batch processing or caching.



100%|██████████| 2932/2932 [2:17:44<00:00,  2.82s/it]  

Generated 2932 descriptions
Length stats - Mean: 173.8, Max: 256


## Encode Tables

In [5]:
# Load encoder
model_name = 'all-MiniLM-L6-v2'
print(f"Loading {model_name}...")
encoder = SentenceTransformer(model_name)
print(f"Dimension: {encoder.get_sentence_embedding_dimension()}")

Loading all-MiniLM-L6-v2...
Dimension: 384


In [6]:
# Encode descriptions
print("Encoding...")
table_embeddings = encoder.encode(
    table_descriptions, batch_size=32, show_progress_bar=True,
    convert_to_numpy=True, normalize_embeddings=True
)
print(f"Shape: {table_embeddings.shape}")

Encoding...


Batches: 100%|██████████| 92/92 [00:06<00:00, 14.72it/s]

Shape: (2932, 384)


## Build FAISS Index

In [7]:
# Build FAISS index
print("Building index...")
index = faiss.IndexFlatIP(encoder.get_sentence_embedding_dimension())
index.add(table_embeddings.astype('float32'))
print(f"✓ Index built with {index.ntotal} tables")

Building index...
✓ Index built with 2932 tables


### Test Custom Queries

In [ ]:
import re

# Test custom queries - change test_query and run
test_query = "olympic medals table"
top_k = 5

print(f"Query: '{test_query}'")
print("=" * 80)

sleep_timer = 1
time.sleep(sleep_timer)

system_msg = 'You are a helpful assistant.'

user_msg = f"""
Given the search query: '{test_query}', generate a description 
of what a relevant table would contain. Describe the table structure, 
columns, and type of data it would have. Keep it under 256 characters.

"""

response = client.chat.completions.create(model="gpt-4o",
messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}],
max_tokens=4000, 
temperature=0.0)

hypothetical_description = response.choices[0].message.content


# Encode and search
test_emb = encoder.encode([hypothetical_description], convert_to_numpy=True, normalize_embeddings=True).astype('float32')
scores_test, indices_test = index.search(test_emb, top_k)

for rank in range(top_k):
    idx = indices_test[0][rank]
    table_id = table_ids[idx]
    row = tables_df[tables_df['table_id'] == table_id].iloc[0]
    
    print(f"\n{rank + 1}. {table_id} (score: {scores_test[0][rank]:.4f})")
    print(f"   Page: {row['page_title']}")
    print(f"   Caption: {row['table_caption']}")
    print(f"   Description: {table_descriptions[idx][:120]}...")
    print("-" * 80)

Query: 'olympic medals table'

1. table-0529-770 (score: 0.7789)
   Page: Lists of Olympic medalists
   Caption: Summer Olympic Games
   Description: This table provides information about the Summer Olympic Games, including the year, host city, number of medal events, t...
--------------------------------------------------------------------------------

2. table-0529-771 (score: 0.7686)
   Page: Lists of Olympic medalists
   Caption: Winter Olympic Games
   Description: This table provides information on the Winter Olympic Games, including the year, links to medal winners and medal tables...
--------------------------------------------------------------------------------

3. table-0952-496 (score: 0.7506)
   Page: List of multiple Olympic gold medalists at a single Games
   Caption: Timeline
   Description: This table lists athletes who won multiple Olympic gold medals at a single Games, detailing the number of golds, the yea...
---------------------------------------------------------

## Evaluation

### Load Queries and Relevance Judgments

In [9]:
# Load queries
queries = {}
with open('data/queries.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(None, 1)
        if len(parts) == 2:
            query_id, query_text = parts
            queries[query_id] = query_text

print(f"Loaded {len(queries)} queries")
print("Examples:", list(queries.items())[:3])

# Load qrels (relevance judgments)
qrels = defaultdict(dict)
with open('data/qrels.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 4:
            query_id, table_id, relevance = parts[0], parts[2], int(parts[3])
            qrels[query_id][table_id] = relevance

qrels = dict(qrels)
print(f"Loaded qrels for {len(qrels)} queries")

Loaded 60 queries
Examples: [('1', 'world interest rates table'), ('2', '2008 beijing olympics'), ('3', 'fast cars')]
Loaded qrels for 60 queries


---

### Generate Hypothetical Descriptions from Queries (HyDE)

In [10]:
query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

print(f"Generating hypothetical table descriptions for {len(query_texts)} queries...")
print("NOTE: This will make ~60 LLM API calls.")
print()

hypothetical_descriptions = []

for query_text in tqdm(query_texts):
    sleep_timer = 1
    time.sleep(sleep_timer)

    system_msg = 'You are a helpful assistant.'

    user_msg = f"""
    Given the search query: '{query_text}', generate a description 
    of what a relevant table would contain. Describe the table structure, 
    columns, and type of data it would have. Keep it under 256 characters.

    """
    response = client.chat.completions.create(model="gpt-4o",
    messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}],
    max_tokens=4000, 
    temperature=0.0)

    hypothetical_description = response.choices[0].message.content

    
    print(f"Hypothetical table description: {hypothetical_description}\n")

    hypothetical_descriptions.append(hypothetical_description)

print(f"Generated {len(hypothetical_descriptions)} hypothetical descriptions")
print(f"\nExamples:")
for i in range(3):
    print(f"  Query: {query_texts[i]}")
    print(f"  Hypothetical: {hypothetical_descriptions[i]}")
    print()

Generating hypothetical table descriptions for 60 queries...
NOTE: This will make ~60 LLM API calls.



  2%|▏         | 1/60 [00:02<02:03,  2.10s/it]

Hypothetical table description: A relevant table would list countries, their central bank interest rates, and the date of the last update. Columns: Country, Interest Rate (%), Last Updated. Data includes country names, numerical rates, and date stamps.



  3%|▎         | 2/60 [00:04<01:59,  2.06s/it]

Hypothetical table description: A relevant table would include columns for Event, Date, Venue, Gold Medalist, Silver Medalist, Bronze Medalist, and Country. It would contain data on the events held, winners, and their respective countries.



...

Hypothetical table description: A relevant table would include columns for Product Name, Review Score, Number of Reviews, Review Summary, and Review Date. It would contain data such as product names, average scores, total reviews, brief summaries, and dates of reviews.



 98%|█████████▊| 59/60 [02:17<00:02,  2.49s/it]

Hypothetical table description: The table would list constellations with columns for Constellation Name, Right Ascension, Declination, and Distance to Earth. Data includes constellation names, celestial coordinates, and proximity in light-years.



100%|██████████| 60/60 [02:19<00:00,  2.33s/it]

Hypothetical table description: A relevant table for 'games age' would include columns: Game Title (text), Release Year (integer), Age Rating (text), Genre (text), and Platform (text). It would list video games, their release years, age ratings, genres, and platforms.

Generated 60 hypothetical descriptions

Examples:
  Query: world interest rates table
  Hypothetical: A relevant table would list countries, their central bank interest rates, and the date of the last update. Columns: Country, Interest Rate (%), Last Updated. Data includes country names, numerical rates, and date stamps.

  Query: 2008 beijing olympics
  Hypothetical: A relevant table would include columns for Event, Date, Venue, Gold Medalist, Silver Medalist, Bronze Medalist, and Country. It would contain data on the events held, winners, and their respective countries.

  Query: fast cars
  Hypothetical: A relevant table for 'fast cars' would include columns like Model, Manufacturer, Top Speed (mph), 0-60 mph Time (se

### Encode Hypothetical Descriptions and Retrieve

In [11]:
# Encode hypothetical descriptions (NOT raw queries)
print(f"Encoding {len(hypothetical_descriptions)} hypothetical descriptions...")
query_embeddings = encoder.encode(
    hypothetical_descriptions, batch_size=32, show_progress_bar=True,
    convert_to_numpy=True, normalize_embeddings=True
)

# Search top 100
k = 100
print(f"Searching top-{k}...")
scores, indices = index.search(query_embeddings.astype('float32'), k)

results = {qid: [table_ids[idx] for idx in indices[i]] for i, qid in enumerate(query_ids)}
print(f"✓ Retrieved {len(results)} query results")

# Show examples
print("\n" + "=" * 80)
print("EXAMPLE RESULTS (HyDE)")
print("=" * 80)
for qid in list(queries.keys())[:3]:
    print(f"\nQuery {qid}: '{queries[qid]}'")
    print(f"Hypothetical: {hypothetical_descriptions[query_ids.index(qid)][:80]}...")
    for rank, tid in enumerate(results[qid][:3], 1):
        rel = qrels.get(qid, {}).get(tid, 0)
        score_val = scores[query_ids.index(qid)][rank-1]
        page = tables_df[tables_df['table_id'] == tid].iloc[0]['page_title']
        print(f"  {rank}. {tid} (score: {score_val:.3f}, rel: {rel}) - {page}")
print("=" * 80)

Encoding 60 hypothetical descriptions...


Batches: 100%|██████████| 2/2 [00:00<00:00,  3.33it/s]

Searching top-100...
✓ Retrieved 60 query results

EXAMPLE RESULTS (HyDE)

Query 1: 'world interest rates table'
Hypothetical: A relevant table would list countries, their central bank interest rates, and th...
  1. table-0552-511 (score: 0.607, rel: 0) - Single deposit
  2. table-0370-614 (score: 0.596, rel: 2) - Eurozone
  3. table-0610-21 (score: 0.543, rel: 0) - Currencies of the European Union

Query 2: '2008 beijing olympics'
Hypothetical: A relevant table would include columns for Event, Date, Venue, Gold Medalist, Si...
  1. table-0109-237 (score: 0.697, rel: 0) - Athletics at the 2007 All-Africa Games
  2. table-0831-590 (score: 0.682, rel: 0) - List of multi-sport events
  3. table-0713-460 (score: 0.681, rel: 0) - List of Olympic medalists in baseball

Query 3: 'fast cars'
Hypothetical: A relevant table for 'fast cars' would include columns like Model, Manufacturer,...
  1. table-1574-853 (score: 0.656, rel: 0) - Suzuki Carry
  2. table-0990-862 (score: 0.619, rel: 1) - Spee

### Calculate Metrics

In [12]:
# Evaluation functions
def recall_at_k(retrieved, relevant, k):
    if len(relevant) == 0:
        return 0.0
    retrieved_at_k = set(retrieved[:k])
    return len(retrieved_at_k & relevant) / len(relevant)

def ndcg_at_k(retrieved, relevance, k):
    if len(relevance) == 0:
        return 0.0
    dcg = sum(relevance.get(retrieved[i], 0) / np.log2(i + 2) for i in range(min(k, len(retrieved))))
    ideal_rels = sorted(relevance.values(), reverse=True)[:k]
    idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_rels))
    return dcg / idcg if idcg > 0 else 0.0

In [13]:
# Evaluate
k_values = [1, 5, 10, 20]
metrics = defaultdict(list)

for query_id, retrieved in results.items():
    if query_id not in qrels:
        continue
    relevance = qrels[query_id]
    relevant = set(tid for tid, rel in relevance.items() if rel > 0)
    
    for k in k_values:
        metrics[f'Recall@{k}'].append(recall_at_k(retrieved, relevant, k))
        metrics[f'nDCG@{k}'].append(ndcg_at_k(retrieved, relevance, k))

# Print results
print("\n" + "="*60)
print("HYDE EVALUATION RESULTS")
print("="*60)
print("\nRecall:")
for k in k_values:
    print(f"  Recall@{k:2d}: {np.mean(metrics[f'Recall@{k}']):.4f}")
print("\nnDCG:")
for k in k_values:
    print(f"  nDCG@{k:2d}  : {np.mean(metrics[f'nDCG@{k}']):.4f}")
print("="*60)


HYDE EVALUATION RESULTS

Recall:
  Recall@ 1: 0.0939
  Recall@ 5: 0.2381
  Recall@10: 0.3520
  Recall@20: 0.5134

nDCG:
  nDCG@ 1  : 0.4917
  nDCG@ 5  : 0.4839
  nDCG@10  : 0.4917
  nDCG@20  : 0.5207


## Inspect Results

In [14]:
# Inspect specific query results
query_id = '1'

print(f"Query {query_id}: {queries[query_id]}")
print(f"Hypothetical description: {hypothetical_descriptions[query_ids.index(query_id)]}")
print("="*80)

for i, table_id in enumerate(results[query_id][:5], 1):
    row = tables_df[tables_df['table_id'] == table_id].iloc[0]
    rel = qrels.get(query_id, {}).get(table_id, 0)
    score_val = scores[query_ids.index(query_id)][i-1]
    
    print(f"\n{i}. {table_id} (score: {score_val:.4f}, relevance: {rel})")
    print(f"   Page: {row['page_title']}")
    print(f"   Section: {row['section_title']}")
    print(f"   Caption: {row['table_caption']}")
    
    try:
        headers = json.loads(row['headers'])
        print(f"   Headers: {headers[:5]}{'...' if len(headers) > 5 else ''}")
    except:
        pass
    
    try:
        sample = json.loads(row['sample_data'])
        print(f"   Sample ({len(sample)} rows): {sample[0][:3]}...")
    except:
        pass
    
    print(f"   LLM Description: {table_descriptions[table_ids.index(table_id)]}")
    print("-" * 80)

Query 1: world interest rates table
Hypothetical description: A relevant table would list countries, their central bank interest rates, and the date of the last update. Columns: Country, Interest Rate (%), Last Updated. Data includes country names, numerical rates, and date stamps.

1. table-0552-511 (score: 0.6070, relevance: 0)
   Page: Single deposit
   Section: Real World Example
   Caption: Real World Example
   Headers: ['Interest Accrued', 'Maturity Amount', 'Gain']
   Sample (1 rows): ['737.62 USD', '5,737.62 USD', '14.75%']...
   LLM Description: This table displays financial data for a single deposit, including the interest accrued, the total maturity amount, and the percentage gain.
--------------------------------------------------------------------------------

2. table-0370-614 (score: 0.5960, relevance: 2)
   Page: Eurozone
   Section: Interest rates
   Caption: Interest rates
   Headers: ['Date', 'Deposit facility', 'Main refinancing operations', 'Marginal lending facil